## Cloning Experience with custom Widget

At the moment ```clone_items()``` will clone ExB application to target Portal, but custom items of Experience still references initial store.

This sample demonstrates cloning complex Portal Item via Item backup and modify properties manually.

In [ ]:
import json
with open('./profiles.json') as f:
    profiles = json.load(f)

In [ ]:
from arcgis.gis import GIS, Item

In [ ]:
source = GIS(**profiles['geodispv9_padmin'])
target = GIS(**profiles['esoapp2_unetwork'])

### Get Experience Builder application with custom Widget

In [ ]:
exb_item_id = '93e980eee73a4450ad9928653c7921ff'
exb_item = Item(source, exb_item_id)
exb_item

### Source portal ExB application

In [ ]:
from arcgis.apps.expbuilder import WebExperience
source_exb = WebExperience(exb_item, gis=source)
source_exb.view()

### Save ExB item configuration to JSON file. Analyze structure.

In [ ]:
exb_config_file = exb_item.download(save_path='MyCustomExB',file_name='source_config.json')
exb_config_file

### Let's modify config file to remap source-target portal items

#### We can access *datasources*, but there is no way to get *widgets* info. So we will modify JSON directly.

In [ ]:
source_exb.datasources

#### Load ExB config

In [ ]:
with open('./MyCustomExB/source_config.json') as f:
    exb_config = json.load(f)

#### Get target Widget and WebMap ItemIds:

In [ ]:
item = target.content.search(query=f"title:ExB Custom Editor widget AND owner:{target.users.me.username}", item_type='Experience Builder Widget')[0]
taget_widget_id = item.id
item = target.content.search(query=f"title:WorkOrders AND owner:{target.users.me.username}", item_type='Web Map')[0]
taget_web_map_id = item.id
print(f'Target Custom Widget ID: {taget_widget_id}')
print(f'Target WebMap ID: {taget_web_map_id}')

#### Get source Widget and WebMap ItemIds:

In [ ]:
source_widget_id = next(widget['itemId'] for key, widget in exb_config['widgets'].items() if widget['label'] == 'Editor')
source_web_map_id = next(ds['itemId'] for key, ds in exb_config['dataSources'].items() if ds['sourceLabel'] == 'WorkOrders')
print(f'Source Custom Widget ID: {source_widget_id}')
print(f'Source WebMap ID: {source_web_map_id}')

#### Replace item Ids and portal urls

In [ ]:
def remap_items(data, val_from, val_to):
    if isinstance(data, list):
        return [remap_items(x, val_from, val_to) for x in data]
    if isinstance(data, dict):
        return {k: remap_items(v, val_from, val_to) for k, v in data.items()}
    return val_to if data == val_from else data # other cases

In [ ]:
new_exb_config = remap_items(exb_config, source_widget_id, taget_widget_id)
new_exb_config = remap_items(new_exb_config, source_web_map_id, taget_web_map_id)
new_exb_config = remap_items(new_exb_config, source.url, target.url)

In [ ]:
exb_config_modified_file = 'MyCustomExB/target_config.json'
with open(exb_config_modified_file, 'w+') as f:
    json.dump(new_exb_config, f)

## Publish remapped Experience to target Portal

In [ ]:
folder_name = 'DevEU24'
folder = next((f for f in target.users.me.folders if f['title'] == folder_name), None)
if not folder:
    folder = target.content.create_folder(folder=folder_name)
folder

In [ ]:
new_exp = WebExperience(path=exb_config_modified_file, gis=target, name='test_clone')
new_exp.datasources

In [ ]:
target_item = new_exp.upload(
    gis = target,
    title = "Custom Editor widget clone", 
    publish = True, 
    auto_remap = True
)
target_item.move(folder=folder)
target_item